In [7]:
#Indicadores precipitacion
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import plotly.graph_objects as go

print("pandas",pd.__version__)
print("numpy",np.__version__)
print("matplotlib",matplotlib.__version__)



pandas 2.2.2
numpy 1.26.4
matplotlib 3.8.4


In [19]:
# Lectura de datos
df_anual = pd.read_excel('M5025_Precipitacion_Anual.xlsx')
df_mensual = pd.read_excel('M5025_Precipitacion_Mensual.xlsx')
df_diario = pd.read_excel('M5025_Precipitacion_Diario.xlsx')
df_horario = pd.read_excel('M5025_Precipitacion_Horario.xlsx')

df_anual['Periodo'] = df_anual['Fecha'].dt.year
df_mensual['Periodo'] = df_mensual['Fecha'].dt.year
df_mensual['Mes'] = df_mensual['Fecha'].dt.month 
df_diario['Periodo'] = df_diario['Fecha'].dt.year

dict_mes = {1:'Ene', 2:'Feb', 3:'Mar', 4:'Abr', 5:'May', 6:'Jun', 7:'Jul', 8:'Ago', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dic'}

In [9]:
# Intensidad máxima horaria
max_value_index = df_horario['Valor'].idxmax()
max_value = df_horario.loc[max_value_index,'Valor']
max_date = df_horario.loc[max_value_index,'Fecha']
# Año más lluvioso
max_year_index = df_anual['Valor'].idxmax()
max_year_value = df_anual.loc[max_year_index, 'Valor']
max_year_date = df_anual.loc[max_year_index, 'Periodo']
# Año más seco
min_year_index = df_anual['Valor'].idxmin()
min_year_value = df_anual.loc[min_year_index, 'Valor']
min_year_date = df_anual.loc[min_year_index, 'Periodo']
# Percentiles diarios
q10 = df_diario.loc[df_diario['Valor']>0,'Valor'].quantile(0.1)
q95 = df_diario.loc[df_diario['Valor']>0, 'Valor'].quantile(0.95)


print(f'Valor Año más seco = {min_year_value}')
print(f'Año más seco = {min_year_date}')
print(f'Valor Año más lluvioso = {max_year_value}')
print(f'Año más lluvioso = {max_year_date}')
print(f'Intensidad máxima acumulada horaria = {max_value}')
print(f'Fecha Intensidad máxima acumulada horaria = {max_date}')
print(f'Percentil 10 = {q10}')
print(f'Percentil 95 = {q95}')

Valor Año más seco = 1115.9
Año más seco = 2013
Valor Año más lluvioso = 1788.8
Año más lluvioso = 2019
Intensidad máxima acumulada horaria = 53.45
Fecha Intensidad máxima acumulada horaria = 2013-05-20 14:00:00
Percentil 10 = 0.3
Percentil 95 = 16.9


In [10]:
# Meses secos y lluviosos

df_max_ind = df_mensual.groupby('Periodo')['Valor'].idxmax()
df_min_ind = df_mensual.groupby('Periodo')['Valor'].idxmin()
df_meses_humedos = df_mensual.loc[df_max_ind.to_list(),['Periodo','Mes','Valor']].reset_index(drop=True)
df_meses_secos = df_mensual.loc[df_min_ind.to_list(),['Periodo','Mes','Valor']].reset_index(drop=True)

In [11]:
# Dias consecutivos secos y lluviosos

umbral = 0.1

 # Clasificar los días que hubo precipitación menor y mayor al umbral
df_diario.loc[df_diario['Valor'] < umbral, 'lluvio'] = 1
df_diario.loc[df_diario['Valor'] >= umbral, 'lluvio'] = 0
# Sumar acumulativamente los días con lluvia
df_diario['cuenta'] = (df_diario['lluvio'] == 0).cumsum()
# Agrupa y sumar acumulativamente los días secos
df_diario['cdd'] = df_diario.groupby('cuenta', dropna=False)['lluvio'].cumsum()
# Extraer los días secos por año
df_dias_secos = df_diario.groupby('Periodo')['cdd'].max()

# Clasificar los días que hubo precipitación menor y mayor a 0.1
df_diario.replace({np.nan: 0}, inplace=True)
df_diario.loc[df_diario['Valor'] <= umbral, 'lluvio'] = 0
df_diario.loc[df_diario['Valor'] > umbral, 'lluvio'] = 1

# Sumar acumulativamente los días secos
df_diario['cuenta'] = (df_diario['lluvio'] == 0).cumsum()
# Agrupa y sumar acumulativamente los días humedos
df_diario['cwd'] = df_diario.groupby('cuenta', dropna=False)['lluvio'].cumsum()
# Extraer los días humedos por año
df_dias_humedos = df_diario.groupby('Periodo')['cwd'].max()




In [23]:

df_mensual['ValorAbs'] = df_mensual.apply(lambda row: abs(row['Valor'] - (df_anual.loc[df_anual['Periodo'] == row['Periodo'], 'Valor'].iloc[0])/12) , axis=1 )
df_indx = df_mensual.groupby('Periodo')[['Valor','ValorAbs']].sum()
df_indx['sindx'] = df_indx.apply(lambda row: round(row['ValorAbs']/row['Valor']*6/11,2), axis=1)

# Tabla de referencia
data = {'Minimo': [0, 0.10, 0.21, 0.32, 0.43, 0.54, 0.65],
        'Maximo': [0.1, 0.21, 0.32, 0.43, 0.54, 0.65,1],
        'Regimen':['Muy homogéneo',
                   'Homogéneo pero con una temporada más húmeda', 
                   'Algo estacional con una temporada seca corta',
                   'Estacional',
                   'Marcadamente estacional con una temporada seca larga',
                   'Mayoría de la lluvia en 3 meses o menos',
                   'Extrema, casi toda la lluvia en 1-2 meses']}

df_ref = pd.DataFrame(data)
df_indx['Interpretacion'] = df_indx.apply(lambda row: df_ref.loc[(df_ref['Minimo']<= row['sindx']) & (row['sindx']<df_ref['Maximo']),'Regimen'].iloc[0], axis=1 )
df_indx.reset_index(inplace=True)


df_resumen_mensual = pd.pivot_table(df_mensual, values='Valor', index='Periodo', columns='Mes', aggfunc='first')
df_resumen_mensual.rename(columns=dict_mes, inplace=True)
df_resumen_mensual = pd.merge(df_resumen_mensual, df_anual.loc[:,['Periodo', 'Valor','Porcentaje de Registro']], on='Periodo')

df_resumen = pd.merge(df_meses_secos, df_meses_humedos, on='Periodo', suffixes=('_Secos', '_Humedos'))
df_resumen = pd.merge(df_resumen, df_dias_humedos, on='Periodo')
df_resumen = pd.merge(df_resumen, df_dias_secos, on='Periodo')
df_resumen = pd.merge(df_resumen, df_indx.loc[:,['Periodo', 'sindx','Interpretacion']], on='Periodo')


print("Valores Mensuales")
print(df_resumen_mensual)

print("Indicadores por Año")
print(df_resumen)

Valores Mensuales
    Periodo     Ene    Feb     Mar    Abr     May     Jun     Jul     Ago  \
0      2012  115.75  156.0   99.75  117.0  148.75  155.75  175.75  150.25   
1      2013     NaN    NaN   87.50   66.0  212.50  102.90  212.00  115.10   
2      2014   99.60   22.5  132.90   94.2  126.80  192.40  158.20  136.70   
3      2015  105.90   90.4   95.90  109.4  107.50  329.80  239.60  142.70   
4      2016   29.50  109.7  127.00  158.7  106.10  249.70  164.50   58.50   
5      2017  157.30   82.1  190.10   88.1  178.40  181.10  260.40  175.90   
6      2018  162.00   66.6   80.60  121.4  198.20  229.50  167.30  177.80   
7      2019  105.40   94.0   67.80   92.6  217.80  281.10  253.40  161.30   
8      2020  118.20   85.1   94.40  111.2  158.10  193.70  256.30  124.00   
9      2021  141.00  145.4  199.20  100.3     NaN     NaN  229.50  118.10   
10     2022     NaN   83.1  137.30  120.5  152.80  199.20  249.50  194.50   
11     2023  139.90   90.7  202.50  149.0  126.10  160.40 